# Worksheet for IGR Selection and BLAST Processing
---
Instructions: Run each cell one at a time following instructions in markdown headers 

In [1]:
%cd /home/jovyan/work

import pandas as pd
import re
import tarfile
import pickle
import plotly as py
import plotly.graph_objs as go
import glob
import os
import shutil
import logging; logging.captureWarnings(True)
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
from src.visualization.visualize import graph_genome, graph_layout
from src.data.rfam_db import rfam_session, Genome
from src.models.predict_model import process_blast
from src.data.command_build import build_infernal_commands

/home/jovyan/work


## Step 1: Set name of the blast results tar.gz below:
---

In [2]:
# Tar file with the results from the blast analysis
import_tar_name = "data/import/Francisella_tularensis_GCA_000008985.1_selection_0.5_2.0_-2.0_blastdata.done.tar.gz"
# Assembly Accession (also the folder the genome's data will be stored in)
assembly_acc = "GCA_000008985.1"
# Selection name (also a subfolder where the data for the selection stored)
selection_name = "selection_0.5_2.0_-2.0"

### Take files from tar.gz and place them in the proper locations in the data/interim folder

In [3]:
data_dir = "data/genomes/{}/{}".format(assembly_acc, selection_name)

def tar_subdir_members(tar, import_tar_name):
    tar_foldername = re.sub(r'(\.done)*\.tar\.gz','',import_tar_name.split('/')[-1]) + '/'
    tar_foldername_length = len(tar_foldername)
    for member in tar.getmembers():
        if member.path.startswith(tar_foldername):
            member.path = member.path[tar_foldername_length:]
            yield member
        
with tarfile.open(import_tar_name, "r:gz") as tar:
    tar.extractall(path=data_dir, members=tar_subdir_members(tar, import_tar_name))

### Rebuild the selection using archived data in the interim folder

In [4]:
# Find the appropriate genome   
session = rfam_session()
genome_list = session.query(Genome).filter(Genome.assembly_acc == assembly_acc).all()
assert len(genome_list) == 1
genome = genome_list[0]

# Read in the annotated igr df from the files
annotated_df = pd.read_csv("{}/data_files/annotated_igrs.csv".format(data_dir), index_col=0)


with open("{}/data_files/svmclf.pickle".format(data_dir), 'rb') as svm_pickle_file:
    svm_clf = pickle.loads(svm_pickle_file.read())

selection = pd.Series(svm_clf.predict(annotated_df.loc[:, ["gc", "log_length"]]))

scatter_plots = graph_genome(annotated_df, selection=selection)
layout = graph_layout(genome)
fig = go.FigureWidget(data=scatter_plots, layout=layout)
fig

Number of known IGRs included:   31 (88.6%)
Number of unknown IGRs included: 108 (7.1%)
Fold Enrichment:  9.67


FigureWidget({
    'data': [{'hoverinfo': 'skip',
              'marker': {'color': 'rgba(192,192,192,0.9)', '…

## Step 2: Process blast results
---
Identifies protein coding regions in the IGR, process_blast calculates an "orf_score" at each nucleotide position. 

This score starts at 0 at each nucleotide position and is increased by $s_i \times w_p \times w_h $ for each overlapping hit. Where 

* $s_i$ is the score_increment (default 2)
* $w_p$ is the poor_score_weight (default 0.5) which is a penalty applied when a hit's blast score is less than the poor_blast_score (default 40).
* $w_h$ is the hypothetical_weight (default 0.5) which is a penalty applied when a hit matches a hypothetical protein.

All nucleotide positions of the IGR where the orf_score $>=$ orf_score_cutoff (default 4) are flagged for removal. Any remaining contiguous IGR portions have their lengths and GC-contents recalculated and checked to see if they still fall in the selection boundaries. 

The decision boundary can be made more flexible by decreasing the svm_decision_cutoff from the default value of 0.0 to a slighly smaller number -0.3. The selection corresponding to this expanded selection boundary will be graphed below.

In [5]:
svm_decision_cutoff=-0.3

fasta_dir = data_dir + '/igr_fastas'
xml_dir = data_dir + '/blast_xml'

process_blast(fasta_dir, xml_dir, svm_clf, poor_blast_score=40, poor_score_weight=0.5, 
              hypothetical_weight=0.5, orf_score_cutoff=4, score_increment=2, 
              svm_decision_cutoff=-0.3)
expanded_selection = svm_clf.decision_function(annotated_df[["gc", "log_length"]]) > svm_decision_cutoff

expanded_scatter_plots = graph_genome(annotated_df, selection=expanded_selection)
expanded_layout = graph_layout(genome)
expanded_layout.title.text = layout.title.text + " (w/ decision cutoff > {})".format(svm_decision_cutoff)
expanded_fig = go.FigureWidget(data=expanded_scatter_plots, layout=expanded_layout)
expanded_fig

Number of known IGRs included:   32 (91.4%)
Number of unknown IGRs included: 130 (8.6%)
Fold Enrichment:  8.56


FigureWidget({
    'data': [{'hoverinfo': 'skip',
              'marker': {'color': 'rgba(192,192,192,0.9)', '…

## Step 3: Build Data and Script tarfiles for Analysis on Cluster
---
`step_dir` refers to the subdirectory where the sto files and scripts will be written for analysis. The script then creates a tarfile in the data/export directory where it can be transferred to a high performance computing cluster to execute the computationally-intensive homology searches. 

In [6]:
step_dir="infernal_step1"

build_infernal_commands(data_dir,step_dir, no_secondary_structure=True)

tarfilename="{}_{}_{}_{}".format('_'.join(genome.scientific_name.split(' ')[0:2]), genome.assembly_acc, selection_name, step_dir)
with tarfile.open("data/export/{}.tar.gz".format(tarfilename), "w:gz") as tar:
    tar.add(data_dir, arcname=tarfilename)
    print("\nTarfle created:",tar.name)


Tarfle created: /home/jovyan/work/data/export/Francisella_tularensis_GCA_000008985.1_selection_0.5_2.0_-2.0_infernal_step1.tar.gz


## Step 4: Transfer Data Tarfile and Run Infernal Script
---
Move Tarfile to your high-performance cluster. Tarfile is located in data/export.
If you typically log into cluster using `ssh`, you'll be able to use `scp` to do the file transfer. Example command: `scp /path/to/dimpl/data/export/tarfilename.tar.gz netid@farnam.hpc.yale.edu:~/project/wherever`.

Untar the uploaded file by using the command `tar xzvf tarfilename`. Remove tarfile using `rm tarfilename.tar.gz` (unpacked files will remain).

Run the Infernal script in the newly extracted folder by entering `./infernal_step1_run.sh` into the terminal.

You can monitor completion of the analysis using the command `squeue -u username` to see running tasks. (PD = pending, R = running)


## Step 5: Create and Transfer Results Tarfile
---
After the all of the jobs complete, run `./make_tar.sh` in the same folder where you ran Infernal.  This will pack up the processed data into a new tarfile called `tarfilename.done.tar.gz`.  

Move the tarfile back to this system and place it in the `data/import` folder.